<a href="https://colab.research.google.com/github/nykolai-d/concreteness-score-of-word/blob/main/GetConcretenessRating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
# Open up workbook
workbook = gc.open_by_url('https://docs.google.com/spreadsheets/d/1-eflh4rGUJdRjJkHKYgzSMWZk6Jaeldl/edit?usp=drive_link&ouid=108068776646159984671&rtpof=true&sd=true')

In [ ]:
# obtain worksheet titles from workbook
sheet_titles = []
for sheet in workbook.worksheets():
  sheet_titles.append(sheet.title)
sheet_titles

['Concreteness_ratings']

In [ ]:
# Get_all_values from sheets and store in dictionary
dict_of_sheets = {}
for sheet_title in sheet_titles:
  sheet = workbook.worksheet(sheet_title)
  values = sheet.get_all_values()
  dict_of_sheets[sheet_title] = values

# To confirm all sheets made it into the dictionary
dict_of_sheets.keys()

dict_keys(['Concreteness_ratings'])

In [ ]:
import pandas as pd
df = pd.DataFrame.from_records(dict_of_sheets.get('Concreteness_ratings'))
df.head(10)

,0,1,2,3,4,5,6,7,8
0,Word,Bigram,Conc.M,Conc.SD,Unknown,Total,Percent_known,SUBTLEX,Dom_Pos
1,sled,0,5,0,0,28,1.00,149,Adjective
2,daisy,0,5,0,1,28,0.96,689,Name
3,peacock,0,5,0,0,26,1.00,261,Name
4,quill,0,5,0,1,29,0.97,139,Name
5,thorn,0,5,0,0,28,1.00,260,Name
6,tulip,0,5,0,0,26,1.00,40,Name
7,angelfish,0,5,0,0,27,1.00,5,Noun
8,antelope,0,5,0,0,28,1.00,50,Noun
9,apple,0,5,0,0,30,1.00,1207,Noun


In [ ]:
new_header = df.iloc[0] #grab the first row for the header
df = df[1:] #take the data less the header row
df.columns = new_header #set the header row as the df header
df.drop(columns = ['Bigram','Conc.SD','Unknown','Total', 'Percent_known','SUBTLEX'],axis = 1, inplace = True)
df

<ipython-input-8-8a18557ab64d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns = ['Bigram','Conc.SD','Unknown','Total', 'Percent_known','SUBTLEX'],axis = 1, inplace = True)


,Word,Conc.M,Dom_Pos
1,sled,5,Adjective
2,daisy,5,Name
3,peacock,5,Name
4,quill,5,Name
5,thorn,5,Name
...,...,...,...
39950,would,1.12,Verb
39951,although,1.07,Conjunction
39952,spirituality,1.07,Noun
39953,eh,1.04,Interjection


### Looping Through Dictionary

In [ ]:
c_ratings = df.to_dict('records')
#c_ratings

In [ ]:
def get_c_rating(word):
  c_rating=[]
  for i in range(len(c_ratings)):
    if(c_ratings[i].get('Word')==word):
      c_rating.append(word)
      c_rating.append(c_ratings[i].get('Conc.M'))
      c_rating.append(c_ratings[i].get('Dom_Pos'))
      break
  return c_rating

In [ ]:
get_c_rating("lollipop")

['lollipop', '4.96', 'Noun']

### Hash Table

In [ ]:
df2 = df.set_index('Word')
df2

,Conc.M,Dom_Pos
Word,,
sled,5,Adjective
daisy,5,Name
peacock,5,Name
quill,5,Name
thorn,5,Name
...,...,...
would,1.12,Verb
although,1.07,Conjunction
spirituality,1.07,Noun


In [ ]:
hash_conc = df2.to_dict('index')
#hash_conc

In [ ]:
def get_conc(word):
  concretness_rating = hash_conc[word].get('Conc.M')
  part_of_speech = hash_conc[word].get('Dom_Pos')
  print("Word: ", word, "\t\nConcreteness Rating: ", concretness_rating, "\t\nPart of Speech: ", part_of_speech)

In [ ]:
get_conc('cheese')

Word:  cheese 	
Concreteness Rating:  4.7 	
Part of Speech:  Noun


In [ ]:
get_conc('dog')

Word:  dog 	
Concreteness Rating:  4.85 	
Part of Speech:  Noun


### Dataframe by Index
Quickest method

In [ ]:
def conc_rating(word):
  return df2.loc[[word]]

In [ ]:
conc_rating('friend')

,Conc.M,Dom_Pos
Word,,
friend,3.07,Noun


In [ ]:
conc_rating('dream')

,Conc.M,Dom_Pos
Word,,
dream,2.6,Noun


In [ ]:
conc_rating('flying')

,Conc.M,Dom_Pos
Word,,
flying,4.1,Verb


In [ ]:
conc_rating('tea')

,Conc.M,Dom_Pos
Word,,
tea,4.69,Noun


### Time

In [ ]:
import time

# looping through dictionary
start_time_ns = time.process_time_ns()
get_c_rating('evil')
end_time_ns = time.process_time_ns()

timer_ns = end_time_ns - start_time_ns
print(timer_ns)

7878612


In [ ]:
# hashtable
start_time_ns = time.process_time_ns()
get_conc('evil')
end_time_ns = time.process_time_ns()

timer_ns = end_time_ns - start_time_ns
print(timer_ns)

Word:  evil 	
Concreteness Rating:  1.93 	
Part of Speech:  Adjective
3697616


In [ ]:
# dataframe by index
start_time_ns = time.process_time_ns()
conc_rating('evil')
end_time_ns = time.process_time_ns()

timer_ns = end_time_ns - start_time_ns
print(timer_ns)

1419166
